John White is the program scheduling manager for the
television channel CCFO. John would like to plan the schedule of television shows for
next Wednesday evening.
The table below lists nine shows under consideration. John must select exactly
five of these shows for the period from 8:00 p.m. to 10:30 p.m. next Wednesday
evening. For each television show, the estimated advertising revenue (in $ millions)
is provided. Furthermore, each show has been categorized into one or more of the
categories ­“Public Interest,” “Violent,” “Comedy,” and “Drama.” In the following
table, a 1 indicates that the show is in the corresponding category and a 0 indicates
it is not.

John would like to determine a revenue-maximizing schedule of television shows
for next Wednesday evening. However, he must be mindful of the following
considerations:

••The schedule must include at least as many shows that are categorized as public
interest as shows that are categorized as violent.

••If John schedules “Loving Life,” then he must also schedule either “Jarred” or
­“Cincinnati Law” (or both).

••John cannot schedule both “Loving Life” and “Urban Sprawl.”

••If John schedules more than one show in the “Violent” category, he will lose an
­estimated $4 million in advertising revenues from family-oriented sponsors.

a. Formulate a binary integer program that models the decisions John faces.

b. Solve the model formulated in part (a). What is the optimal revenue?

In [2]:
# Show            | Revenue | Public Interest | Violent | Comedy | Drama
#                 |         |                 |         |        |
# Sam’s Place     | $6      |  0              |    0    |    1   |     1
# Texas Oil       | $10     |  0              |    1    |    0   |     1
# Cincinnati Law  | $9      |  1              |    0    |    0   |     1
# Jarred          | $4      |  0              |    1    |    0   |     1
# Bob & Mary      | $5      |  0              |    0    |    1   |     0
# Chainsaw        | $2      |  0              |    1    |    0   |     0
# Loving Life     | $6      |  1              |    0    |    0   |     1
# Islanders       | $7      |  0              |    0    |    1   |     0
# Urban Sprawl    | $8      |  1              |    0    |    0   |     0

shows = ["Sam’s Place", "Texas Oil", "Cincinnati Law", "Jarred", "Bob & Mary", "Chainsaw", "Loving Life", "Islanders", "Urban Sprawl"]
revenue = [6,10,9,4,5,2,6,7,8]
public_interest = [0,0,1,0,0,0,1,0,1]
violent = [0,1,0,1,0,1,0,0,0]
comedy = [1,0,0,0,1,0,0,1,0]
drama = [1,1,1,1,0,0,1,0,0]

In [3]:
from docplex.mp.model import Model

m = Model()

X = m.binary_var_list(shows)
XX_violent = m.binary_var(name = 'ViolentShowsMoreThanOne')

public_interest_count = sum([public_interest[i] * X[i] for i in range(len(X))])
violent_count = sum([violent[i] * X[i] for i in range(len(X))])
m.add_constraint(public_interest_count >= violent_count)

m.add_constraint(X[6] <= X[2] + X[3])
m.add_constraint(X[6] * X[8] <= 0)

m.add_indicator(XX_violent, violent_count >= 2, 1)
m.add_indicator(XX_violent, violent_count <= 1, 0)

m.maximize(sum([revenue[i] * X[i] for i in range(len(X))]) - (XX_violent * 4))

In [4]:
solution = m.solve(log_output = True)

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1


Tried aggregator 2 times.
MIQCP Presolve eliminated 1 rows and 4 columns.
MIQCP Presolve modified 1 coefficients.
Aggregator did 1 substitutions.
Reduced MIQCP has 9 rows, 13 columns, and 27 nonzeros.
Reduced MIQCP has 6 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQCP has 1 quadratic constraints.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0       49.9443     0                     57.0000        1         
      0     0       49.1803     2                     Cone: 1        2         
      0     0       48.6679     3                     Cone: 2        3         
      0     0       48.5700     3    

In [5]:
print(solution)

solution for: docplex_model2
objective: 45
status: OPTIMAL_SOLUTION(2)
Sam’s Place=1
Texas Oil=1
Cincinnati Law=1
Jarred=1
Bob & Mary=1
Islanders=1
Urban Sprawl=1
ViolentShowsMoreThanOne=1

